In [3]:
import torch
import numpy as np
from torch import nn
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import torch.optim as optim
from torch.nn import Sequential, ReLU, Linear
from sklearn.metrics import roc_auc_score

device = "cuda" if torch.cuda.is_available() else "cpu"


### Preparação do dataset

In [6]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]
print(data)
data.train_mask = data.val_mask = data.test_mask = None
print(data)


transform = T.RandomLinkSplit(is_undirected=True,add_negative_train_samples=True)
train_data, val_data, test_data = transform(data)

train_data = train_data.to(device)
val_data=val_data.to(device)
test_data = test_data.to(device)

# test_data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708])


### Definição do modelo

In [26]:
class Feedforward(torch.nn.Module):
    
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            
            self.input_size = input_size
            self.hidden_size  = hidden_size
            
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.fc2 = torch.nn.Linear(self.hidden_size, self.hidden_size)
            self.fc3 = torch.nn.Linear(self.hidden_size, self.hidden_size)
            self.fc4 = torch.nn.Linear(self.hidden_size, 1)
            
            self.relu = torch.nn.ReLU()
            self.out_act = nn.Sigmoid()
            
            
        def forward(self, x):
            output = self.fc1(x)
            output = self.relu(output)
            
            output = self.fc2(output)
            output = self.relu(output)
            
            output = self.fc3(output)
            output = self.relu(output)
            
            output = self.fc4(output)
            
            output = self.out_act(output)

            return output

### Criando features que representam as arestas: soma das features dos nós

In [21]:
num_features = train_data.x.shape[1]

## Gerando as features para o treino
featuresEdges_treino = ""

totalEdges_treino = train_data.edge_label_index.shape[1]

for col in range(0,totalEdges_treino):
    node1 = train_data.edge_label_index[0,col]
    node2 = train_data.edge_label_index[1,col]
    
    vals1 = train_data.x[[node1,]]
    vals1 = torch.reshape(vals1, (1, num_features))
    
    vals2 = train_data.x[[node2,]]
    vals2 = torch.reshape(vals2, (1, num_features))
    
    if col == 0:
        featuresEdges_treino = vals1+vals2
        continue
        
    somado = vals1+vals2
    
    featuresEdges_treino =  torch.cat((featuresEdges_treino, somado), dim=0) 


##########################################################################################
##Gerando as features para o teste
featuresEdges_teste = ""

totalEdges_teste = test_data.edge_label_index.shape[1]
print(totalEdges_teste)
for col in range(0,totalEdges_teste):

    node1 = test_data.edge_label_index[0,col]
    node2 = test_data.edge_label_index[1,col]
    
    vals1 = test_data.x[[node1,]]
    vals1 = torch.reshape(vals1, (1, num_features))

    vals2 = test_data.x[[node2,]]
    vals2 = torch.reshape(vals2, (1, num_features))
    
    if col == 0:
        featuresEdges_teste = vals1+vals2
        continue
        
    somado = vals1+vals2
    
    featuresEdges_teste =  torch.cat((featuresEdges_teste, somado), dim=0) 

featuresEdges_teste.shape

2110


torch.Size([2110, 1433])

### Instância do modelo e outros parâmetros

In [62]:
num_features = train_data.x.shape[1]
hidden_channel = 64

#input = (número de features), e hidden size = 10 (número de neurôneos na camada escondida)
model = Feedforward(num_features, hidden_channel).to(device)
print(model)
criterion = torch.nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

Feedforward(
  (fc1): Linear(in_features=1433, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (out_act): Sigmoid()
)


### Treino e teste

In [63]:
model.eval()
y_pred = model(featuresEdges_treino)
train_data.edge_label = torch.reshape(train_data.edge_label,(-1,1))

antes_treino = criterion(y_pred, train_data.edge_label) 

print('Teste - perda antes do treinamento' , antes_treino.item())

model.train()
epoch = 1000

for epoch in range(epoch):
    optimizer.zero_grad()
    
    # Passe Forward
    y_pred = model(featuresEdges_treino)
    
    # Computa a perda
    loss = criterion(y_pred, train_data.edge_label)
    
    print('Epoch {}: perda treino: {}'.format(epoch, loss.item()))
    
    # Passe de Backward
    loss.backward()
    optimizer.step()


Teste - perda antes do treinamento 0.6944001317024231
Epoch 0: perda treino: 0.6944001317024231
Epoch 1: perda treino: 0.6896529197692871
Epoch 2: perda treino: 0.6757488250732422
Epoch 3: perda treino: 0.6507232785224915
Epoch 4: perda treino: 0.6165645718574524
Epoch 5: perda treino: 0.5855273008346558
Epoch 6: perda treino: 0.5854929685592651
Epoch 7: perda treino: 0.5627989768981934
Epoch 8: perda treino: 0.5632736682891846
Epoch 9: perda treino: 0.5038442611694336
Epoch 10: perda treino: 0.5090611577033997
Epoch 11: perda treino: 0.4494154751300812
Epoch 12: perda treino: 0.431398868560791
Epoch 13: perda treino: 0.4035916030406952
Epoch 14: perda treino: 0.3546677529811859
Epoch 15: perda treino: 0.33595922589302063
Epoch 16: perda treino: 0.2856478691101074
Epoch 17: perda treino: 0.2637999355792999
Epoch 18: perda treino: 0.23194469511508942
Epoch 19: perda treino: 0.19474805891513824
Epoch 20: perda treino: 0.17337380349636078
Epoch 21: perda treino: 0.13761265575885773
Epoch 

Epoch 180: perda treino: 5.833375689690001e-05
Epoch 181: perda treino: 5.720705303247087e-05
Epoch 182: perda treino: 5.612508539343253e-05
Epoch 183: perda treino: 5.508233152795583e-05
Epoch 184: perda treino: 5.4065254516899586e-05
Epoch 185: perda treino: 5.305963713908568e-05
Epoch 186: perda treino: 5.205262277740985e-05
Epoch 187: perda treino: 5.1048205932602286e-05
Epoch 188: perda treino: 5.006243009120226e-05
Epoch 189: perda treino: 4.911549331154674e-05
Epoch 190: perda treino: 4.82190735056065e-05
Epoch 191: perda treino: 4.737402923638001e-05
Epoch 192: perda treino: 4.656937744584866e-05
Epoch 193: perda treino: 4.579011874739081e-05
Epoch 194: perda treino: 4.501891817199066e-05
Epoch 195: perda treino: 4.424925646162592e-05
Epoch 196: perda treino: 4.34839676017873e-05
Epoch 197: perda treino: 4.27299746661447e-05
Epoch 198: perda treino: 4.199907198199071e-05
Epoch 199: perda treino: 4.1298786527477205e-05
Epoch 200: perda treino: 4.062649531988427e-05
Epoch 201: pe

Epoch 354: perda treino: 1.4955062397348229e-06
Epoch 355: perda treino: 1.4878276033414295e-06
Epoch 356: perda treino: 1.4802782288825256e-06
Epoch 357: perda treino: 1.4731321016370202e-06
Epoch 358: perda treino: 1.4659776752523612e-06
Epoch 359: perda treino: 1.4588803196602385e-06
Epoch 360: perda treino: 1.451500224902702e-06
Epoch 361: perda treino: 1.444572035325109e-06
Epoch 362: perda treino: 1.437660216652148e-06
Epoch 363: perda treino: 1.4306593811852508e-06
Epoch 364: perda treino: 1.4239249139791355e-06
Epoch 365: perda treino: 1.4172067039908143e-06
Epoch 366: perda treino: 1.4103673038334819e-06
Epoch 367: perda treino: 1.4024959682501503e-06
Epoch 368: perda treino: 1.3921813888373435e-06
Epoch 369: perda treino: 1.3809152505928068e-06
Epoch 370: perda treino: 1.3705681567444117e-06
Epoch 371: perda treino: 1.3585521401182632e-06
Epoch 372: perda treino: 1.34410083774128e-06
Epoch 373: perda treino: 1.3252710004962864e-06
Epoch 374: perda treino: 1.3014175692660501e-

Epoch 537: perda treino: 3.465119746692835e-08
Epoch 538: perda treino: 3.451409469334976e-08
Epoch 539: perda treino: 3.4336675724944143e-08
Epoch 540: perda treino: 3.41673214165894e-08
Epoch 541: perda treino: 3.401409287562274e-08
Epoch 542: perda treino: 3.386892899470695e-08
Epoch 543: perda treino: 3.370764289911676e-08
Epoch 544: perda treino: 3.357054367825185e-08
Epoch 545: perda treino: 3.3425379797336063e-08
Epoch 546: perda treino: 3.329634168380835e-08
Epoch 547: perda treino: 3.3135052035504486e-08
Epoch 548: perda treino: 3.297376949262798e-08
Epoch 549: perda treino: 3.282860561171219e-08
Epoch 550: perda treino: 3.2643125535969375e-08
Epoch 551: perda treino: 3.2530216742543416e-08
Epoch 552: perda treino: 3.2360865986902354e-08
Epoch 553: perda treino: 3.222377031875112e-08
Epoch 554: perda treino: 3.2102796865274286e-08
Epoch 555: perda treino: 3.198990228270304e-08
Epoch 556: perda treino: 3.177216356675672e-08
Epoch 557: perda treino: 3.165925477333076e-08
Epoch 5

Epoch 720: perda treino: 1.7982186406584333e-08
Epoch 721: perda treino: 1.793380199899275e-08
Epoch 722: perda treino: 1.790154513514608e-08
Epoch 723: perda treino: 1.7788650552574836e-08
Epoch 724: perda treino: 1.7756395465085006e-08
Epoch 725: perda treino: 1.7699946397442545e-08
Epoch 726: perda treino: 1.7667687757239037e-08
Epoch 727: perda treino: 1.7603177582259377e-08
Epoch 728: perda treino: 1.7554793174667793e-08
Epoch 729: perda treino: 1.749834765973901e-08
Epoch 730: perda treino: 1.737738841711689e-08
Epoch 731: perda treino: 1.7329005785882146e-08
Epoch 732: perda treino: 1.728062137829056e-08
Epoch 733: perda treino: 1.7199981883209148e-08
Epoch 734: perda treino: 1.7167726795719318e-08
Epoch 735: perda treino: 1.7119342388127734e-08
Epoch 736: perda treino: 1.7054828660434396e-08
Epoch 737: perda treino: 1.700644602919965e-08
Epoch 738: perda treino: 1.6974189165352982e-08
Epoch 739: perda treino: 1.692580298140456e-08
Epoch 740: perda treino: 1.685322992273086e-08


Epoch 892: perda treino: 1.170841379405374e-08
Epoch 893: perda treino: 1.1692285362130406e-08
Epoch 894: perda treino: 1.1676157818385491e-08
Epoch 895: perda treino: 1.1643902730895661e-08
Epoch 896: perda treino: 1.1627773410793907e-08
Epoch 897: perda treino: 1.1611645867048992e-08
Epoch 898: perda treino: 1.1595518323304077e-08
Epoch 899: perda treino: 1.1539071920196875e-08
Epoch 900: perda treino: 1.152294437645196e-08
Epoch 901: perda treino: 1.1498752172656168e-08
Epoch 902: perda treino: 1.1466497973344758e-08
Epoch 903: perda treino: 1.1418116230288433e-08
Epoch 904: perda treino: 1.1385861142798603e-08
Epoch 905: perda treino: 1.1361669827181231e-08
Epoch 906: perda treino: 1.1337477623385439e-08
Epoch 907: perda treino: 1.1321350079640524e-08
Epoch 908: perda treino: 1.130522164771719e-08
Epoch 909: perda treino: 1.127296656022736e-08
Epoch 910: perda treino: 1.1248775244609988e-08
Epoch 911: perda treino: 1.1232647700865073e-08
Epoch 912: perda treino: 1.1216518380763318e

In [64]:
model.eval()

y_pred = model(featuresEdges_teste)
y_pred = torch.reshape(y_pred,(-1,1))

threshold = torch.tensor([0.5])

#Atribui 0 ou 1 de acordo com o threshold
results = (y_pred>threshold).float()*1

#reshape
test_data.edge_label = torch.reshape(test_data.edge_label,(-1,1))

#Area under the curve - AUC
roc_auc_score(test_data.edge_label.detach().numpy(),  results.detach().numpy())

0.757345971563981